## Introduction
In this notebook, we will creat a simple agent with Strands SDK and use one of the Strands Community Tools 

In [ ]:
# Run this cell to install the required packages if you haven't already done so.
%pip install -q  strands-agents-tools

In [ ]:
## execute this cell to restart kernel 
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [ ]:
# Create OpenAI Client with Nova API
from openai import OpenAI
import os
from dotenv import load_dotenv
import json

#Set up your environment variables
api_key = os.getenv("NOVA_API_KEY")
base_url = "https://api.nova.amazon.com/v1"

# Create OpenAI client
client = OpenAI(api_key=api_key, base_url=base_url)

# Configure model_id
model_id = 'nova-2-lite-v1' # You can change this to any other Nova model available to you

In [ ]:
# By default, certain tools that perform potentially sensitive operations (like file modifications, shell commands, or code execution)
# will prompt for user confirmation before executing. 
# To bypass these confirmation prompts, you can set the BYPASS_TOOL_CONSENT environment variable (which we will do here programmatically).
os.environ["BYPASS_TOOL_CONSENT"] = "true"

In [ ]:
## Instantiate the Strands Agent's HTTP Tool for API calls
from strands import Agent
from strands_tools import http_request

agent = Agent(tools=[http_request])

message_json = {
  "model": model_id,
  "messages": [
    {
      "role": "user",
      "content": "evaluate 7**6 by using the calculator tool "
    }],
  "tool_choice": "auto",
  "stream": "False"
}

# Make a POST request
response = agent.tool.http_request(
    method="POST",
    url="https://api.nova.amazon.com/v1/chat/completions",
    headers={"Content-Type": "application/json"},
    body=json.dumps(message_json),
    auth_type="Bearer",
    auth_token=api_key
)


In [ ]:
response['status']

In [ ]:
## Post process raw http response and print the story
raw_response = response['content'][2]['text']
raw_response = raw_response.replace('Body: ', '')
json_r = json.loads(raw_response)
print(json_r['choices'][0]['message']['content'])

In [ ]:
print("Model Response: \n")
print(json.dumps(json_r, indent=4))